<a href="https://colab.research.google.com/github/guilhermegranchopro/Prometheus/blob/main/Prometheus/Copy_of_Alpaca3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar os requeriemntos do Alpaca API

In [39]:
pip install alpaca-trade-api

Uso o Yahoo Finance API para saber quando é que a empresa que estou a usar entrou no mercado, quando a empresa IPO.

In [40]:
import yfinance as yf

# Defina o símbolo da empresa
ticker_symbol = 'AAPL'  # Exemplo com Apple

# Obtenha os dados históricos
ticker_data = yf.Ticker(ticker_symbol)
historical_data = ticker_data.history(period='max')

# Obtenha a data da primeira cotação
first_trading_date = historical_data.index[0]
print(f"A empresa {ticker_symbol} começou a ser cotada na bolsa em:", first_trading_date.strftime("%Y-%m-%d"))

def get_start_date(symbol):
    ticker_data = yf.Ticker(symbol)
    historical_data = ticker_data.history(period='max')
    first_trading_date = historical_data.index[0]
    return first_trading_date

A empresa AAPL começou a ser cotada na bolsa em: 1980-12-12


Saber o dia que é hoje, o dia que o código está a correr, o modelo está a ser criado

In [41]:
import pandas as pd
from datetime import datetime, timedelta, timezone

# Obter a data e hora atuais
current_date_time = datetime.now()

# Formatar a data como string
current_date = current_date_time.strftime("%Y-%m-%d")
print(f"Hoje é: {current_date}")

def get_end_date(symbol):
    # Obter a data e hora atuais
    current_date_time = datetime.now()
    # Make start timezone aware
    current_date_time = current_date_time.replace(tzinfo=timezone.utc)
    return current_date_time

Hoje é: 2025-01-27


Usas a tua senha da tua conta no ALpca API para usares a API, tens de te conectar á tua conta.

In [42]:
import alpaca_trade_api as tradeapi
import pandas as pd
import time
from datetime import datetime, timedelta

def alpaca_autentification():
    # Replace with your Alpaca API credentials
    API_KEY = 'AK1RX6F8W6QX207XPLDF'
    SECRET_KEY = 'WaPoTTxkQBGzC51LajCdyw8Pl6svbINa9eDu9TMK'
    BASE_URL = 'https://api.alpaca.markets'  # Use 'https://api.alpaca.markets' for live trading

    # Initialize the Alpaca API
    api = tradeapi.REST(API_KEY, SECRET_KEY, BASE_URL, api_version='v2')

    return api

Depois chamas a API para retirar os dados que necessitas. Para a API não te bloquear usas um intervalo de 1 segundo sempre que chamas a API para conseguires atingir elevada granuralidade dos dados, sem seres bloqueado pela API.

In [43]:
# Function to fetch data in chunks to avoid rate limits
def fetch_data(symbol, start, end, timeframe, api):
    all_data = []
    current_start = start

    while current_start < end:
        current_end = current_start + timedelta(days=7)
        if current_end > end:
            current_end = end

        # Fetch the historical data
        bars = api.get_bars(
            symbol,
            timeframe,
            start=current_start.strftime('%Y-%m-%d'),
            end=current_end.strftime('%Y-%m-%d')
        ).df

        all_data.append(bars)
        current_start = current_end
        time.sleep(1)  # Adjust sleep interval based on rate limits

    return pd.concat(all_data)

Instalas a biblioteca necessária para calculares o garu necessário de semelhança entre strings

In [44]:
pip install python-Levenshtein

Instalas a biblioteca necessária para calculares o garu necessário de semelhança entre strings

In [45]:
pip install fuzzywuzzy

Connecta com a tua google drive

In [46]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


função para criar o nome do ficheiro. O nome do ficheiro encriptará todas varaiveis/hyperparametros necessários para voltar a correr o ficheiro do modelo ou o csv com todos os dados. (tens um dicionário do que cada uma dos paramteros representa mais a baixo)

In [47]:
def create_file_name(symbol, max_period, start_date, end_date,
                     timeframe, fisrt_marker, second_marker, third_marker,
                     n_rows, sort, relative_first_marker, relative_second_marker,
                     relative_third_marker, decimals, LSTM_boolean, epochs,
                     early_stopping, call_back, patience, batch_size,
                     test_loss, test_accuracy, model_file):

    # Nome aproximado do arquivo
    if model_file == True:
        nome_aproximado = 's=' + symbol + '+' + 'mp=' + str(max_period) + '+' + 'sd=' + \
        start_date + '+' + 'ed=' + end_date + '+' + 'tf=' + timeframe + '+' + \
        'fm=' + fisrt_marker + '+' + 'sm=' + second_marker + '+' + \
        'tm=' + third_marker + '+' + 'r=' + str(n_rows) + '+' + \
        'sort=' + str(sort) + '+' + 'rfm=' + str(relative_first_marker) + '+' + \
        'rsm=' + str(relative_second_marker) + '+' + \
        'rtm=' + str(relative_third_marker) + '+' + \
        'd=' + decimals + '+' + 'Lb=' + str(LSTM_boolean) + '+' + 'e=' + str(epochs) + '+' + \
        'es=' + str(early_stopping) + '+' + 'cb=' + call_back + '+' + \
        'p=' + str(patience) + '+' + 'bs=' + str(batch_size) + '+' + \
        'tl=' + str(test_loss) + '+' + 'ta=' + str(test_accuracy) + '.keras'
    else:
        nome_aproximado = 's=' + symbol + '+' + 'mp=' + str(max_period) + '+' + 'sd=' + \
        start_date + '+' + 'ed=' + end_date + '+' + 'tf=' + timeframe + '.csv'

    return nome_aproximado

função para procurar se o modelo e os dados são originais, caso não sejamm originais é porque foi encontrado dados e pelo menos um modelo identicos ao que está a ser pedido ao algoritmo já guradado na base de dados e caso isso aconteça, os dados não seram novamente retirados da API mas sim seram usados aqueles que já se encontram na base de dados (arquivos da tua google drive). Isto é feito por compração de nomes de ficheiros, já que no nome dos ficheiros encotra-se encriptado tudo o que é necessário para voltar a correr o algoritmo em termos de hyperparamtros e variaveis.

In [48]:
import os
from fuzzywuzzy import process

def old_dataset_function(symbol, max_period, start_date, end_date,
                timeframe, fisrt_marker, second_marker, third_marker,
                n_rows, sort, relative_first_marker, relative_second_marker,
                relative_third_marker, decimals, LSTM_boolean, epochs,
                early_stopping, call_back, patience, batch_size,
                test_loss, test_accuracy, model_file):

    # Defina o caminho do diretório no Google Drive
    diretorio = '/content/drive/MyDrive/AI Financial Analisys/Summer Project/Dataset/Raw Data'

    # Nome aproximado do arquivo
    nome_aproximado = create_file_name(symbol, max_period, start_date, end_date,
                                      timeframe, fisrt_marker, second_marker, third_marker,
                                      n_rows, sort, relative_first_marker, relative_second_marker,
                                      relative_third_marker, decimals, LSTM_boolean, epochs,
                                      early_stopping, call_back, patience, batch_size,
                                      test_loss, test_accuracy, model_file)

    # Lista todos os arquivos no diretório
    arquivos = os.listdir(diretorio)

    # Encontra o arquivo mais próximo
    if process.extractOne(nome_aproximado, arquivos):
        arquivo_mais_proximo, pontuacao = process.extractOne(nome_aproximado, arquivos)
    else:
        arquivo_mais_proximo = None
        pontuacao = None

    if arquivo_mais_proximo and pontuacao==100:
        caminho_do_modelo = os.path.join(diretorio, arquivo_mais_proximo)
        print("Nome do arquivo mais próximo:")
        print(arquivo_mais_proximo)
        print()
        print("Caminho completo do modelo:")
        print(caminho_do_modelo)
        print()
        print("Pontuação de similaridade:")
        print(pontuacao)
        old_dataset = True
    else:
        print("Nenhum arquivo próximo encontrado.")
        caminho_do_modelo = ''
        old_dataset = False

    return old_dataset, caminho_do_modelo

Usando a alpaca API para sacar os dados agora que já deste login á tua conta da alpaca com a tua senha pessoal anteriormente.

In [49]:
def get_all_data(symbol = 'AAPL', max_period = True, start_date = '', end_date= '', timeframe = '1Min'):
    api = alpaca_autentification()

    if max_period and start_date == '' and end_date == '':
        start_date = get_start_date(symbol)
        end_date = get_end_date(symbol)
    elif start_date != '' and end_date != '':
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    else:
        start_date = datetime(2022, 12, 31)
        end_date = datetime(2023, 12, 31)

    # Fetch the historical data
    apple_data = fetch_data(symbol, start_date, end_date, timeframe, api)
    return apple_data

Esta função irá preparar dos dados para o modelo. Do Alpca seram retirados multiplos parametros do mercado interessa-te selecionar apenas os que queres alimentar ao modelo para isso esat função é usada para os selecionar. podes escolher entre 'close', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap', que são os parametros de mercado dados por default pelo Alpaca. Mas para além disso crias te outras 2 métricas com base nas métricas já dadas caso as queiras testar eslas são: 'average_oc' e 'average_hl', que correspondem, respetivamente, ao ponto médio entre o open e o close e o ponto médio entre o high e o low.



*   close é o preço de fecho da ação naquele intervalo temporal
*   high é o preço mais alto da ação naquele intervalo temporal
*   low é o preço mais baixo da ação naquele intervalo temporal
*   trade_count é o número de matches entre compradosres e vendedores naquele intervalo temporal
*   open é o preço de abertura da ação naquele intervalo temporal
*   vwap é o preço da ação por volume naquele intervalo temporal ( uma forma diferente de olhar para o valor intrinseco de uma ação)



In [50]:
# Function to prepare the data
def prepare_data(data, fisrt_marker, second_marker, third_marker):
    # Selecionar colunas relevantes e fazer uma cópia para evitar advertências
    data_selected = data[['close', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap']].copy()

    # Calcular a média entre 'Open' e 'Close' usando .loc para evitar o SettingWithCopyWarning
    data_selected.loc[:, 'average_oc'] = data_selected[['open', 'close']].mean(axis=1)

    # Calcular a média entre 'High' e 'Low' usando .loc para evitar o SettingWithCopyWarning
    data_selected.loc[:, 'average_hl'] = data_selected[['high', 'low']].mean(axis=1)

    # Selecionar as colunas finais
    if third_marker != '':
        data_final = data_selected[[fisrt_marker, second_marker, third_marker]]
    else:
        data_final = data_selected[[fisrt_marker, second_marker]]

    return data_final

Agora temos de continuara a preparação dos dados para serem alimentado ao modelo para isso crio matrizes para segmentar os dados, cada matriz vai ter uma label: 0 ou 1. Assim o dataset vai ser constituido por ,atrizes de tamanho a defenir pelo o usuário do algoritmo e pelas suas respetivas labels de 0 ou 1. As matrizes no máximo poderão ter até 3 colunas, cada coluna representa uma metrica difrente do mercado. As linhas estam inicialmente organizadas de forma sequencial e representação a cada uma a granuralidade mínima do dataset, assim se a granuralidade é de 1 minuto e uma matriz tem 5 linhas, cada matriz representa 5 minutos continuos de métricas do mercado no tempo. As métricas de cada matriz por coluna podem ser formatadas como relativas, o que significa que as métricas da matriz seram apresentadas de forma relativa á matriz em questão, o que pode ser interessante para modelos sem ser um LSTM, pois permite que os dados históricos sejam baralhados temporalmente e manetenham coerência. Como é o caso por exemplo do preço e do volume, com a inflação é normal que o preço aumente mas não queremos que isso cause uma precepção errada de magnitude superior ao nosso modelo assim o 'relative' faz com que as métricas das matrizes apareçam como valores ponderados aos restantes valores da matriz, assim cada valor da métrica relativa é dividida pelo somatório dos valores dessa métrica presentes na matriz.

In [51]:
import numpy as np
from sklearn.preprocessing import StandardScaler

def create_matrices(data_final, n = 12, fisrt_marker = 'volume', second_marker = 'vwap', third_marker = '', relative_fisrt_marker = True, relative_second_marker = False, relative_third_marker = False):
    num_rows = data_final.shape[0]
    num_matrices = num_rows // n  # Número de matrizes completas que podemos formar

    matrices = []
    matrices_y = []
    scaler = StandardScaler()

    for i in range(num_matrices):
        start_idx = i * n
        end_idx = start_idx + n
        matrix = data_final.iloc[start_idx:end_idx].copy()

        if relative_fisrt_marker:
            # Calcular a soma da coluna de volume
            matrix[fisrt_marker] = scaler.fit_transform(matrix[[fisrt_marker]]).flatten()

            # Converter para valores numpy e adicionar à lista de matrizes
            #matrices.append(matrix.values)

        if relative_second_marker:
            # Calcular a soma da coluna de volume
            matrix[second_marker] = scaler.fit_transform(matrix[[second_marker]]).flatten()

            # Converter para valores numpy e adicionar à lista de matrizes
            #matrices.append(matrix.values)

        if relative_third_marker:
            # Calcular a soma da coluna de volume
            matrix[third_marker] = scaler.fit_transform(matrix[[third_marker]]).flatten()

            # Converter para valores numpy e adicionar à lista de matrizes
        matrices.append(matrix.values)

        # Criar labels
        if i > 0:  # Ignorar a primeira matriz para alinhamento correto
            first_value = matrices[i][0, 1]
            last_value = matrices[i][n-1, 1]
            delta = last_value - first_value
            delta_bi = 1 if delta >= 0 else 0
            matrices_y.append(delta_bi)

    return matrices, matrices_y

Caso se entenda que a ordem temporal, devido á elevada granuraridade dos dados de uma matriz, é irrelevante, esta função permite ordenar por ordem decrescente da métrica da segunda coluna os valores de uma matriz e assim por exemplo caso na segunda coluna se tenha o valor de volume estaria se a observar em cada matriz o preço por ordem crescente do volume. Esta função pode ser ativada ou desativada e pode ser interessante para uma elevada granuralidade em que as matrizes apenas representam 5 minutos e não se está a usar uma LSTM.

In [52]:
def sort_matrices(matrices):
    # Ordenar as linhas de cada matriz por ordem decrescente da coluna Average (segunda coluna)
    matrices_x = []
    for matrix in matrices:
        sorted_indices = np.argsort(matrix[:, 1])[::-1]
        sorted_matrix = matrix[sorted_indices]
        matrices_x.append(sorted_matrix)
    return matrices_x

Funçaõ para passar as matrizes para tensores para poderem ser tartadas pelo TensorFlow

In [53]:
def tensor_creator(matrices_x, matrices_y):
    # Converter listas para arrays numpy
    matrices_x = np.array(matrices_x)
    matrices_y = np.array(matrices_y)
    return matrices_x, matrices_y

Função que integra todas as outras funções que falamos para obter o dataset e manipulá-lo conforme o utilizador do algoritmo deseja

In [54]:
def get_dataset(company_data, n_rows, sort = True, fisrt_marker = 'volume', second_marker = 'vwap' ,
                third_marker = '', relative_fisrt_marker = True, relative_second_marker = False,
                relative_third_marker = False, decimals = ''):
    matrices_x = []
    matrices_y = []

    matrices_x, matrices_y = create_matrices(company_data, n_rows, fisrt_marker, second_marker, third_marker, relative_fisrt_marker, relative_second_marker, relative_third_marker)

    if sort:
        matrices_x = sort_matrices(matrices_x)

    # Remover a última matriz de matrices_x para alinhamento com matrices_y
    if matrices_x:
        matrices_x.pop()

    matrices_x, matrices_y = tensor_creator(matrices_x, matrices_y)

    if decimals != '':
        matrices_x = np.round(matrices_x, decimals = decimals)
    return matrices_x, matrices_y

Esta função divide o dataset em treino, validação e teste. Esta divisão tem de ser feita de forma diferente caso se esteja a usar um LSTM ou não. Se for usado um LSTM para manter a ordem temporal dos dados o datset é dividido em fatias sequêncis primeiros 60% dos dados históricos para treino, proximos 20% para validação e ultimos 20% para teste. Se não for usado uma LSTM os dados históricos são formatados em matrizes, as matrizes são baralhadas e entregues baralhadas tempralmente ao modelo.

In [55]:
from sklearn.model_selection import train_test_split

def split_dataset(matrices_x, matrices_y, LSTM = True):

    if LSTM:
        # Total de amostras
        n_samples = matrices_x.shape[0]

        # Índices para divisão
        train_size = int(0.6 * n_samples)  # 60% para treino
        val_size = int(0.2 * n_samples)    # 20% para validação
        test_size = n_samples - train_size - val_size  # 20% para teste

        # Dividir os dados sequencialmente
        X_train = matrices_x[:train_size]
        y_train = matrices_y[:train_size]

        X_val = matrices_x[train_size:train_size + val_size]
        y_val = matrices_y[train_size:train_size + val_size]

        X_test = matrices_x[train_size + val_size:]
        y_test = matrices_y[train_size + val_size:]
    else:
        # Dividir os dados em conjuntos de treinamento + validação e teste
        X_train_val, X_test, y_train_val, y_test = train_test_split(matrices_x, matrices_y, test_size=0.2, random_state=42)

        # Dividir o conjunto de treinamento + validação em treinamento e validação
        X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

    return X_train, X_val, X_test, y_train, y_val, y_test


Função para criar o modelo de AI. Esta função permite escolher entre duas arquiteturas diferentes. É possível escolher entre usar uma LSTM, caso se queira dar prioridade á lógica temporal do mercado, ou usar uma arquitetura diferente caso não se queira dar essa relevância à ordem temporal.

Pode ser interessante não usar uma LSTM. O padrão de procedimento na industria é usar um LSTM devido ao motivo lógico que esxite uma padrão temporal de acontecimentos obvios no mercado. No entanto, para explorar outras prespetivas de olhar para o mercado, desenvolves te várias ferramentas que em teoria permitiriam olhar para o mercado sem ser relevante a ordem temporal. A ideia é que pela teoria do mercado eficiente combiando com a tua ideia de um torque financeiro, que deve abrir um curto intervalo de tempo no mercado para ganhar lucro comprativamento com os outros players do mercado. Assim se aumentares por muito a granuralidade e assim estarás a fazer previsões para a matriz seguinte, ou seja, se as matrizes apenas representam 5 minutos no mercado, logo somando essa a ideia de uma granuralidade elevada com o facto de tornar os parametros na matriz relativos, e ordenar as linhas da matriz por ordem crecente dos valores e não tempora, todas estas ferramentas juntas em teoria permitiriam analisar o mercado sem ter em conta a ordem temporal.


In [56]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import warnings

# Função para construir a rede neural
def build_model(input_shape, LSTM_boolean=True):
    warnings.filterwarnings('ignore', category=UserWarning, module='keras')
    model = Sequential()

    if LSTM_boolean:
        model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(64, return_sequences=False))
        model.add(Dropout(0.2))
    else:
        model.add(Flatten(input_shape=input_shape))
        model.add(Dense(64, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))

    # Compilando o modelo com uma taxa de aprendizado ajustada
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    model.summary()

    return model

Função para treinar o modelo, usamos earlystopping para saber qunado para de forma mais correta as épocas. Esta função também grava o histórico de treinamento assim como também testa o modelo contra o dataset de teste e grava os resultados.

In [57]:
def model_training(model, X_train, X_val, X_test, y_train, y_val, y_test, epochs=5, early_stopping = True, call_back = 'val_accuracy', patience = 2, batch_size=32):

    # Treinar o modelo
    print("Treinando o modelo...")

    if early_stopping:
        early_stopping = EarlyStopping(monitor=call_back, patience=patience, restore_best_weights=True)
        history = model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, validation_data=(X_val, y_val), callbacks=[early_stopping])
    else:
        history = model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, validation_data=(X_val, y_val))

    print()

    # Avaliar o modelo nos dados de teste
    print("Avaliação no conjunto de teste:")
    test_loss, test_accuracy = model.evaluate(X_test, y_test)

    return model, history, test_loss, test_accuracy

função que permite dar plot dos resultados da historia de treinamento e dar plot aos resultado do modelo no dataset de teste.

In [58]:
# Plotar a precisão e a perda ao longo das épocas
import matplotlib.pyplot as plt

def graphics_analysis(history):

    # Precisão
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    # Perda
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

Todas as variaveis que se podem perdsonalizar no algoritmo.

As primeiras possibilidades de cada variável são aquelas as quais inicialmnete pensas-te que faziam mais sentido:

* symbol = 'AAPL', (retira do ficheito execel, tens o código que extrai)
* max_period = True, (podes ter erros por excesso de pedidos do API)
* start_date = '', '2022-10-31', (1 ano funciona bem para test runs)
* end_date = '', '2023-10-31', (1 ano funciona bem para test runs)
* timeframe = '1Min', '5Min', (mais de 5Min parece perder sentido a teoria)
* fisrt_marker = 'volume', 'close', 'high', 'low', 'trade_count', 'open', 'vwap', 'average_oc', 'average_hl', (a ideia inicial é ver o volume relativo, mas pode ser interessante estudar o trade_count)
* second_marker = 'vwap', 'volume', 'close', 'high', 'low', 'trade_count', 'open', 'average_oc', 'average_hl', (a ideia inicial era ver a média entre o high e o low - 'average_hl', mas fica na duvida ver a média entre o open e o close - 'average_oc', pode também ser muito interessante investigar o vwap)
* third_marker = '', 'volume', 'close', 'high', 'low', 'trade_count', 'open', 'vwap', 'average_oc', 'average_hl', (não pensas te em usar uma terceira coluna para criar as matrizes, mas se os resultados impiricos assim o provarem usa)
* n_rows = 12, 10, 15, 30, 5, (pensa no timeframe quando fazes isso, o 12 pareceu ser o melhor em testes impiricos)
* sort = True, (tens de pensar na sequencia temporal se faz sentido manter ou não)
* relative_first_marker = True, (tens de pensar em que marker correspondente estás a usar, pensa se matemáticamente faz sentido ou não colocares o marcador com valores relativos para todo o tensor, faltam te dados impiricos, o volume pensas te inicialmnete deixar em impirico para o tensor para normalizares os tensores)
* relative_second_marker = False, (tens de pensar em que marker correspondente estás a usar, pensa se matemáticamente faz sentido ou não colocares o marcador com valores relativos para todo o tensor, faltam te dados impiricos)
* relative_third_marker = False, (tens de pensar em que marker correspondente estás a usar, pensa se matemáticamente faz sentido ou não colocares o marcador com valores relativos para todo o tensor, faltam te dados impiricos)
* decimals = '', 3, 4 (tens de ver o que os dados impiricos te dizem se com arrendondamentos facilitas ou n a vida do AI)
* LSTM_boolean = True, (em caso de falso não usas um LSTM e partes do precipio que a relação causal deixa de ser importante para a cosntrução do tensor, quando normalizas os dados a esperança seria de não teres de usar um LSTM)
* epochs = 5, (este numero tende a ser irrelevante porque tens um checkpoint de callback referente ao melhor resultado de validation accuracy)
* early_stopping = True, (podes ligar ou desligar de forma automática o early_stopping, ou seja, o call back aqui)
* call_back = 'val_accuracy', 'val_loss', 'accuracy', 'loss' (escolhes o paramtero que monitora o call back)
* patience = 2, 50, 100, 500, 1000 (escolhes a paciencia do call back)
* batch_size = 64, 16, 32, 128, 256, 512 (varias com o tramanho do dataset, mas pensa que tens conseguido datasets bem grandes portanto usa os maiores valores)

In [59]:
symbol = 'AAPL'
max_period = False
start_date = '2016-01-1'
end_date = '2024-12-30'
timeframe = '5Min'
fisrt_marker = 'vwap'
second_marker = 'trade_count'
third_marker = ''
n_rows = 36
sort = False
relative_first_marker = True
relative_second_marker = False
relative_third_marker = False
decimals = ''
LSTM_boolean = True
epochs = 500
early_stopping = True
call_back = 'val_accuracy'
patience = 100
batch_size = 128

Função que agrega todas as anteriores para preceber se com base nas variaveis introduzidas no algoritmo, o algoritmo já foi corrido alguma vez com estes settings ou não. Caso tenha sido ele vai buscar á google drive os dados guardados anteriormente com estes settings o que otimiza o tempo que demora a correr o algoritmo, pois caso contrário é necessário usar a API Alpaca para obter todos os dados o que pode demorar bastante tempo.

In [60]:
import pandas as pd

old_dataset, caminho_do_modelo = old_dataset_function(symbol, max_period, start_date, end_date,
                                  timeframe, fisrt_marker, second_marker, third_marker,
                                  n_rows, sort, relative_first_marker, relative_second_marker,
                                  relative_third_marker, decimals, LSTM_boolean, epochs,
                                  early_stopping, call_back, patience, batch_size, 0, 0, False)

if old_dataset:
    company_data = pd.read_csv(caminho_do_modelo, index_col=0)
else:
    company_data = get_all_data(symbol, max_period, start_date, end_date, timeframe)

print(company_data)
print(company_data.shape)
print()

Nome do arquivo mais próximo:
s=AAPL+mp=False+sd=2016-01-1+ed=2024-12-30+tf=5Min.csv

Caminho completo do modelo:
/content/drive/MyDrive/AI Financial Analisys/Summer Project/Dataset/Raw Data/s=AAPL+mp=False+sd=2016-01-1+ed=2024-12-30+tf=5Min.csv

Pontuação de similaridade:
100
                              close      high       low  trade_count  \
timestamp                                                              
2016-01-01 00:00:00+00:00  105.4000  105.4300  105.4000            8   
2016-01-01 00:10:00+00:00  105.4100  105.4100  105.4000            4   
2016-01-01 00:15:00+00:00  105.4000  105.4100  105.4000            7   
2016-01-01 00:35:00+00:00  105.4700  105.4700  105.4000           10   
2016-01-01 00:45:00+00:00  105.5099  105.5100  105.4800           13   
...                             ...       ...       ...          ...   
2024-12-31 00:30:00+00:00  251.8689  251.8700  251.8300           33   
2024-12-31 00:40:00+00:00  251.8682  251.8682  251.8682           18   
20

Esta função é chamada para fazer o tratamento dos dados brutos conforme as settings que o usuário decidiu antes dos dados serem alimentados ao modelo.

In [61]:
company_data_prepared = prepare_data(company_data, fisrt_marker, second_marker, third_marker)

print(company_data_prepared)
print(company_data_prepared.shape)
print()

                                 vwap  trade_count
timestamp                                         
2016-01-01 00:00:00+00:00  105.407123            8
2016-01-01 00:10:00+00:00  105.404289            4
2016-01-01 00:15:00+00:00  105.404203            7
2016-01-01 00:35:00+00:00  105.444645           10
2016-01-01 00:45:00+00:00  105.498727           13
...                               ...          ...
2024-12-31 00:30:00+00:00  251.863500           33
2024-12-31 00:40:00+00:00  251.868200           18
2024-12-31 00:45:00+00:00  251.850000           10
2024-12-31 00:50:00+00:00  251.845227           44
2024-12-31 00:55:00+00:00  251.858417           60

[462418 rows x 2 columns]
(462418, 2)



esta função separa os dadaos em matrizes com as dimensões esplicitas nas settings do algoritmo.

In [62]:
matrices_x, matrices_y = get_dataset(company_data_prepared, n_rows, sort, fisrt_marker, second_marker , third_marker, relative_first_marker, relative_second_marker, relative_third_marker, decimals)

print(matrices_x)
print(matrices_x.shape)
print(matrices_y)
print(matrices_y.shape)
print()

[[[ 1.96366184e+00  8.00000000e+00]
  [ 1.95989113e+00  4.00000000e+00]
  [ 1.95977671e+00  7.00000000e+00]
  ...
  [-7.45908799e-01  8.00000000e+00]
  [-8.99052097e-01  3.60000000e+01]
  [-1.09359864e+00  3.20000000e+01]]

 [[ 7.52706781e-01  8.00000000e+00]
  [ 4.35276939e-01  5.00000000e+00]
  [-1.67857421e-01  2.20000000e+01]
  ...
  [-3.18415715e+00  1.64890000e+04]
  [-4.17136931e-01  1.31950000e+04]
  [ 5.71827795e-01  1.10080000e+04]]

 [[-1.28929401e+00  7.24500000e+03]
  [-1.06559743e+00  6.47400000e+03]
  [-1.42673499e+00  5.37600000e+03]
  ...
  [ 1.06004938e+00  2.95000000e+03]
  [ 1.56403518e+00  4.00500000e+03]
  [ 2.18549797e+00  3.90000000e+03]]

 ...

 [[ 6.08264161e-01  1.72000000e+02]
  [ 5.09302396e-01  1.11000000e+02]
  [ 4.48243405e-01  4.00000000e+01]
  ...
  [-2.54161775e+00  4.60000000e+02]
  [-2.97111258e+00  2.20000000e+02]
  [-3.07564050e+00  2.77000000e+02]]

 [[ 2.47397573e+00  2.30000000e+03]
  [ 1.22059014e+00  2.43000000e+02]
  [ 1.13934931e+00  2.4600

Esta função divide o dataset em treino, teste e validação

In [63]:
X_train, X_val, X_test, y_train, y_val, y_test = split_dataset(matrices_x, matrices_y, LSTM_boolean)

# Exibir as formas dos arrays resultantes
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)
print()

Shape of X_train: (7705, 36, 2)
Shape of y_train: (7705,)
Shape of X_val: (2568, 36, 2)
Shape of y_val: (2568,)
Shape of X_test: (2570, 36, 2)
Shape of y_test: (2570,)



Este parte do código permite obter as características da arqitetura do AI que está a ser usada.

In [64]:
if third_marker == '':
  input_shape = (n_rows, 2)
else:
  input_shape = (n_rows, 3)
model = build_model(input_shape, LSTM_boolean)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 36, 64)              │          17,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 36, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 50,241 (196.25 KB)

 Trainable params: 50,241 (196.25 KB)

 Non-trainable params: 0 (0.00 B)

Esta função treina o modelo nos dados de treino e validação. E testa o modelo contra os dados de teste.

In [ ]:
model, history, test_loss, test_accuracy = model_training(model, X_train, X_val, X_test, y_train, y_val, y_test, epochs, early_stopping, call_back, patience, batch_size)

Treinando o modelo...
Epoch 1/500
61/61 ━━━━━━━━━━━━━━━━━━━━ 13s 146ms/step - accuracy: 0.7482 - loss: 0.5332 - val_accuracy: 0.8088 - val_loss: 0.4670
Epoch 2/500
61/61 ━━━━━━━━━━━━━━━━━━━━ 6s 105ms/step - accuracy: 0.8158 - loss: 0.4383 - val_accuracy: 0.7858 - val_loss: 0.4665
Epoch 3/500
61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 144ms/step - accuracy: 0.8242 - loss: 0.4262 - val_accuracy: 0.8127 - val_loss: 0.4619
Epoch 4/500
61/61 ━━━━━━━━━━━━━━━━━━━━ 6s 105ms/step - accuracy: 0.8260 - loss: 0.4260 - val_accuracy: 0.8150 - val_loss: 0.4453
Epoch 5/500
61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 144ms/step - accuracy: 0.8283 - loss: 0.4070 - val_accuracy: 0.7843 - val_loss: 0.4652
Epoch 6/500
61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 126ms/step - accuracy: 0.8185 - loss: 0.4210 - val_accuracy: 0.8224 - val_loss: 0.4405
Epoch 7/500
61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 104ms/step - accuracy: 0.8304 - loss: 0.4018 - val_accuracy: 0.8119 - val_loss: 0.4405
Epoch 8/500
61/61 ━━━━━━━━━━━━━━━━━━━━ 12s 129ms/step - accuracy: 0.8356 -

Plota os resultados durante a fase de treinamento e durante a fase de teste.

In [ ]:
graphics_analysis(history)

Caso o modelo seja corrido em settings originais ou caso o modelo apresente resultados inovadores o modelo será gravado usando esta função e os settings que influenciram o modelo e os resultados obtidos pelo modelo são encriptados no nome do ficheiro no qual o modelo fica guardado.

In [ ]:
import os

# Defina o caminho do diretório no Google Drive
caminho_para_salvar_modelo = '/content/drive/MyDrive/AI Financial Analisys/Summer Project/Neural Network/Models'

# Verifique se o diretório existe, caso contrário, crie-o
if not os.path.exists(caminho_para_salvar_modelo):
    print(f"O dirétório {caminho_para_salvar_modelo} não existe.")

file_name = create_file_name(symbol, max_period, start_date, end_date,
                     timeframe, fisrt_marker, second_marker, third_marker,
                     n_rows, sort, relative_first_marker, relative_second_marker,
                     relative_third_marker, decimals, LSTM_boolean, epochs,
                     early_stopping, call_back, patience, batch_size,
                     test_loss, test_accuracy, True)

caminho_completo_modelo = os.path.join(caminho_para_salvar_modelo, file_name)

# Salva o modelo em formato
model.save(caminho_completo_modelo)

print("Ficheiro do Modelo:")
print(file_name)
print()
print("Modelo salvo em:")
print(caminho_para_salvar_modelo)

Caso os dados sejam coletados por uma versão do algoritmo que correu em settings originais os dados seram gravados usando esta função e os settings que influenciram o levaram a obeter os respetivos dados/dataset modelo são encriptados no nome do ficheiro CSV no qual os dados ficam guardados.

In [ ]:
import os

# Defina o caminho do diretório no Google Drive
caminho_para_salvar_dados = '/content/drive/MyDrive/AI Financial Analisys/Summer Project/Dataset/Raw Data'

# Verifique se o diretório existe, caso contrário, crie-o
if not os.path.exists(caminho_para_salvar_dados):
    os.makedirs(caminho_para_salvar_dados)
    print(f"Diretório {caminho_para_salvar_dados} criado com sucesso.")
else:
    print(f"O diretório {caminho_para_salvar_dados} já existe.")

print()

file_name = create_file_name(symbol, max_period, start_date, end_date,
                     timeframe, fisrt_marker, second_marker, third_marker,
                     n_rows, sort, relative_first_marker, relative_second_marker,
                     relative_third_marker, decimals, LSTM_boolean, epochs,
                     early_stopping, call_back, patience, batch_size,
                     test_loss, test_accuracy, False)

# Defina o caminho completo para salvar o arquivo
caminho_completo_dados = os.path.join(caminho_para_salvar_dados, file_name)

# Salve o dataframe no caminho especificado
company_data.to_csv(caminho_completo_dados)

print("Ficheiro dos Dados:")
print(file_name)
print()
print("Dados salvos em:")
print(caminho_para_salvar_dados)